In [1]:
from bokeh.io import output_notebook, output_file, show, save
from bokeh.plotting import figure
from bokeh.layouts import row
from bokeh.palettes import Dark2_5 as palette
from bokeh.models import Range1d

import pandas as pd
import numpy as np
from glob import glob

# output_notebook()
output_file('pareto.html')

In [2]:
filenames = glob('logs/*.csv')
get_name = lambda fname: fname.split('-')[0].split('_', maxsplit=1)[1]
model_names = list(set(map(get_name, filenames)))

x_col = 'p50 latency'
y_col = 'Inferences/Second'
batch_size = 8
min_y, max_y = 1<<25, 0

figures = []
for i, instances in enumerate([1, 2, 4]):
    p = figure(
        height=500,
        width=350,
        tools='',
        title='{} Concurrent Model Instances'.format(instances)
    )
    p.xaxis.axis_label = 'p50 Latency (us)'
    p.yaxis.axis_label = 'Throughput (Samples/Second)'

    for model_name, color in zip(model_names, palette):
        # first one isn't full for some reason, going to just hack
        csvs = glob('logs/deepclean_{}-{}-*.csv'.format(model_name, instances))[1:]

        data = pd.read_csv(csvs[0], usecols=[x_col, y_col]).values
        for csv in csvs[1:]:
            df = pd.read_csv(csv, usecols=[x_col, y_col])
            data += df.values
        data /= len(csvs)
        y, x = data.T
        p.line(
            x,
            batch_size*y,
            line_color=color,
            legend_label=model_name,
            line_width=2.3
        )

        min_y = min(min_y, batch_size*y.min())
        max_y = max(max_y, batch_size*y.max())

    p.legend.location = 'bottom_right'
    figures.append(p)

for p in figures:
    p.y_range = Range1d(0.95*min_y, 1.05*max_y)

# show(row(figures))
save(row(figures))

'C:\\Users\\agunny\\Documents\\mit-logs\\pareto.html'